<a href="https://colab.research.google.com/github/The237/DeepLearningCourses/blob/main/01_06_Regression_Lineaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header = None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
data = data
target = target

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
data.shape

(506, 13)

In [13]:
target.shape

(506,)

In [14]:
X = data

In [19]:
y = target.reshape((len(data),1))

# Train test split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 0)

In [21]:
X_train.shape

(379, 13)

In [22]:
X_test.shape

(127, 13)

# Standard scaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)